# Homework 10
#### Course Notes
**Language Models:** https://github.com/rjenki/BIOS512/tree/main/lecture17  
**Unix:** https://github.com/rjenki/BIOS512/tree/main/lecture18  
**Docker:** https://github.com/rjenki/BIOS512/tree/main/lecture19

## Question 1
#### Make a language model that uses ngrams and allows the user to specify start words, but uses a random start if one is not specified.

#### a) Make a function to tokenize the text.

In [14]:
install.packages(c(
  "httr",
  "jsonlite",
  "tokenizers",
  "stringr",
  "R6",
  "digest",
  "viridis"
))


The downloaded binary packages are in
	/var/folders/yh/x49cjjwn6w1c2sjv6g2r8ktm0000gn/T//RtmplUt6jk/downloaded_packages


In [15]:
library(httr)
library(tokenizers)

tokenize_text <- function(text) {
    tokenizers::tokenize_words(text, lowercase=TRUE, strip_punct=TRUE)[[1]]
}

#### b) Make a function generate keys for ngrams.

In [16]:
key_from <- function(ngram, sep = "\x1f") {
    paste(ngram, collapse=sep)
}

#### c) Make a function to build an ngram table.

In [17]:
build_ngram_table <- function(tokens, n, sep = "\x1f") {
    if (length(tokens) < n) return(new.env(parent = emptyenv()))
    tbl <- new.env(parent = emptyenv())
    for (i in seq_len(length(tokens) - n + 1L)) {
        ngram <- tokens[i:(i + n - 2L)]
        next_word <- tokens[i + n - 1L]
        key <- paste(ngram, collapse = sep)
        counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
        if (next_word %in% names(counts)) {
            counts[[next_word]] <- counts[[next_word]] + 1L
        } else {
            counts[[next_word]] <- 1L
        }
        tbl[[key]] <- counts
    }
    tbl
}

#### d) Function to digest the text.

In [18]:
digest_text <- function(text, n) {
    tokens <- tokenize_text(text)
    build_ngram_table(tokens, n)
}

#### e) Function to digest the url.

In [19]:
digest_url <- function(url, n) {
    res <- httr::GET(url)
    txt <- httr::content(res, as = "text", encoding = "UTF-8")
    digest_text(txt,n)
}

#### f) Function that gives random start.

In [20]:
random_start <- function(tbl, sep = "\x1f") {
    keys <- ls(envir = tbl, all.names=TRUE)
    if (length(keys)==0) stop("No n-grams available. Digest text first.")
    picked <- sample(keys, 1)
    strsplit(picked, sep, fixed=TRUE)[[1]]
}

#### g) Function to predict the next word.

In [21]:
predict_next_word <- function(tbl, ngram, sep = "\x1f") {
    key <- paste(ngram, collapse = sep)
    counts <- if(!is.null(tbl[[key]])) tbl[[key]] else integer(0)
    if (length(counts) == 0) return(NA_character_)
    sample(names(counts), size=1, prob=as.numeric(counts))
}

#### h) Function that puts everything together. Specify that if the user does not give a start word, then the random start will be used.

In [23]:
make_ngram_generator <- function(tbl, n, sep = "\x1f") {
    force(tbl); n <- as.integer(n); force(sep)
    function(start_words = NULL, length = 10L) {
        if ((is.null(start_words)) || length(start_words) != n - 1L) {
            start_words <- random_start(tbl, sep=sep)
        }
        word_sequence <- start_words
        for (i in seq_len(max(0L, length - length(start_words)))) {
            ngram <- tail(word_sequence, n - 1L)
            next_word <- predict_next_word(tbl, ngram, sep=sep)
            if (is.na(next_word)) break
            word_sequence <- c(word_sequence, next_word)
        }
        paste(word_sequence, collapse= " ")
    }
}

## Question 2
#### For this question, set `seed=2025`.
#### a) Test your model using a text file of [Grimm's Fairy Tails](https://www.gutenberg.org/cache/epub/2591/pg2591.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [27]:
set.seed(2025)
url <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"
tbl3 <- digest_url(url, n=3)
gen3 <- make_ngram_generator(tbl3, n=3)
output <- gen3(start_words = c("the", "king"), length = 15)
cat(output)

the king has forbidden me to marry another husband am not i shall ride upon

In [37]:
set.seed(2025)
url <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"
tbl3 <- digest_url(url, n=3)
gen3 <- make_ngram_generator(tbl3, n=3)
print(gen3(length=15))

[1] "spread the jam over it spread its wings and crying here comes our hobblety jib"


#### b) Test your model using a text file of [Ancient Armour and Weapons in Europe](https://www.gutenberg.org/cache/epub/46342/pg46342.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [33]:
set.seed(2025)
url <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt"
tbl3 <- digest_url(url, n=3)
gen3 <- make_ngram_generator(tbl3, n=3)
output <- gen3(start_words = c("the", "king"), length = 15)
cat(output)

the king he added to the entire exclusion of the swords were made prisoners the

In [38]:
set.seed(2025)
url <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt"
tbl3 <- digest_url(url, n=3)
gen3 <- make_ngram_generator(tbl3, n=3)
print(gen3(length=15))

[1] "lamentation de lemburn came forth completely armed after the fashion of this may be seen"


#### c) Explain in 1-2 sentences the difference in content generated from each source.

Using the n=3 model with a specificed start word, constrains the text to a particular context. However, without using a start word, the text produces a random start with unique sequences. The difference in words used it based on the difference in material and stories. 

## Question 3
#### a) What is a language learning model? 
#### b) Imagine the internet goes down and you can't run to your favorite language model for help. How do you run one locally?

A language learning model is a model that predicts the following word in a sequence based on the context and using probability. 

OLLAMA allows you to run language models locally. 

## Question 4
#### Explain what the following vocab words mean in the context of typing `mkdir project` into the command line. If the term doesn't apply to this command, give the definition and/or an example.
| Term | Meaning |  
|------|---------|
| **Shell** | the program that interprets the commands and executes them |
| **Terminal emulator** | the application that applies a text interface to interact with the shell |
| **Process** | an instance of a running program  |
| **Signal** | a message sent to a process to tell it to do something  |
| **Standard input** | the default way a process receives input |
| **Standard output** | the default destination for output |
| **Command line argument** | extra information passed to a command |
| **The environment** | a set of variables and settings available to process |

## Question 5
#### Consider the following command `find . -iname "*.R" | xargs grep read_csv`.
#### a) What are the programs?
#### b) Explain what this command is doing, part by part.

a) the programs include find, xargs and grep 

b) first, the command is searching for files in the directory looking for files ending in .R and is taking the output of this to send it to the command xargs, which is reading the list of .R files and is converting them into arguments to grep, which then searches for the text "read_csv" within each file and outputs them.

## Question 6
#### Install Docker on your machine. See [here](https://github.com/rjenki/BIOS512/blob/main/lecture18/docker_install.md) for instructions. 
#### a) Show the response when you run `docker run hello-world`.
#### b) Access Rstudio through a Docker container. Set your password and make sure your files show up on the Rstudio server. Type the command and the output you get below.
#### c) How do you log in to the RStudio server?

a) Hello from Docker!
This message shows that your installation appears to be working correctly.

b) docker run -d -p 8787:8787 \
  -e PASSWORD=christou3 \
  -v /Users/angelchristou/my-first-binder:/home/rstudio/files \
  rocker/rstudio

e177d270eea239b10ca594c804603975009f0b25d56c6c337663a66f7274664d
docker: Error response from daemon: failed to set up container networking: driver failed programming external connectivity on endpoint tender_mahavira (0483afac67ad56eaa0abaf7bf6eb4ca70723802cc7086cc752e3f39de6059c07): Bind for 0.0.0.0:8787 failed: port is already allocated

Run 'docker run --help' for more information

c) type rstudio for the username and your personalized password. 